## Ising Model Simulation and Bayesian Inference

## Libraries

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

## Sampling from Ising Model

### Functions

In [ ]:
# Define the Ising model energy function
def spin_energy(spins, i,j, N, J, h):
    E_interact = 0
    E_field = 0

    target_spin = spins[i,j]

    neigh_coords = [(i-1, j), (i, j-1), (i+1, j), (i, j+1)]
    neigh_coords = [coord for coord in neigh_coords 
                    if (coord[0] in range(N) and coord[1] in range(N))]
    
    neigh_spins = [spins[coord[0], coord[1]] for coord in neigh_coords]

    for neigh_spin in neigh_spins:
        E_interact -= J * target_spin * neigh_spin   # Nearest neighbor interactions
    
    E_field -= h * spins[i, j]  # External field

    return E_interact, E_field


def total_energy(spins, N, J, h):

    for i in range(N):
        for j in range(N):
            E_interact, E_field = spin_energy(spins, i, j, N, J, h)

    return E_interact + E_field

# Metropolis-Hastings sampling
def metropolis_sampling(spins, N, J, h, T, num_steps):
    for step in range(num_steps):
        i, j = np.random.randint(N, size=2)

        delta_E = (
            2 * J * spins[i, j] * (spins[(i+1)%N, j] 
            + spins[i, (j+1)%N] + spins[(i-1)%N, j] 
            + spins[i, (j-1)%N]) + 2 * h * spins[i, j])
        
        if delta_E <= 0 or np.random.rand() < np.exp(-delta_E / T):
            spins[i, j] *= -1

    return spins

In [ ]:
# Define the Ising model parameters
N = 20  # Lattice size
J = 1.0  # Interaction coefficient
h = 0.0  # External field
T = 2.0  # Temperature

In [ ]:

# Initialize the spin configuration
spins = np.random.choice([-1, 1], size=(N, N))

# Define the Ising model energy function
def energy(spins):
    E = 0.0
    for i in range(N):
        for j in range(N):
            E -= J * spins[i, j] * (spins[(i+1)%N, j] + spins[i, (j+1)%N])  # Nearest neighbor interactions
            E -= h * spins[i, j]  # External field
    return E

# Metropolis-Hastings sampling
def metropolis_sampling(spins, T, num_steps):
    for step in range(num_steps):
        i, j = np.random.randint(N, size=2)
        delta_E = 2 * J * spins[i, j] * (spins[(i+1)%N, j] + spins[i, (j+1)%N] + spins[(i-1)%N, j] + spins[i, (j-1)%N]) + 2 * h * spins[i, j]
        if delta_E <= 0 or np.random.rand() < np.exp(-delta_E / T):
            spins[i, j] *= -1

    return spins

# Run the simulation
num_steps = 10000
spins = metropolis_sampling(spins, T, num_steps)

# Plot the spin configuration
plt.figure(figsize=(6, 6))
plt.imshow(spins, cmap=cm.bwr, vmin=-1, vmax=1)
plt.colorbar()
plt.show()